# Big Data Modeling and Management Assigment - Homework 1

# Submission

GROUP MEMBERS:

|STUDENT NAME|STUDENT NUMBER|
|---|---|
|Gaspar Pereira|20230984|
|Íris Moreira|20240659|
|Jude Gbenimako|20240700|
|Rita Wang|20240551|

## 🍺 The Beer project  🍺 

As it was shown in classes, graph databases are a natural way of navegating related information. For this first project we will be taking a graph database to analyse beer and breweries!   

The project datasets are based on [kaggle](https://www.kaggle.com/ehallmar/beers-breweries-and-beer-reviews), released by Evan Hallmark. 

### Problem description

Imagine you are working in the Data Management department of Analytics company.
Explore the database via python neo4j connector and/or the graphical tool in the NEO4J webpage. Answer the questions while adjusting the database to meet the needs of your colleagues.
Please record and keep track of your database changes, and submit the file with all cells run and with the output shown.

### Questions

1. Explore the database: get familiar with current schema, elements and other important database parameters. [1 point]
2. Adjust the database and mention reasoning behind: e.g. clean errors, remove redundancies, adjust schema as necessary. Visualize the final version of database schema. [4 points]
3. Analytics department requires the following information for the biweekly reporting: [5 points]
    1. How many reviews has the beer with the most reviews?
    2. Which three users wrote the most reviews about beers?
    3. Find all beers that are described with following words: 'fruit', 'complex', 'nutty', 'dark'.
    4. Which top three breweries produce the largest variety of beer styles?
    5. Which country produces the most beer styles?
4. Market Analysis department in your company accesses and updates the trends data on the daily basis. Given that, consider how you need to optimize the database and its performance so that the following queries are efficient. Measure performance to communicate your improvements using PROFILE before final query. Answer the following: [4 points]
    1. Using ABV score, find five strongest beers, display their ABV score and the corresponding brewery? Keep in mind that the strongest known beer is Snake Venom, and deal with the error entries in the database.
    2. Using the answer from question 2, find the top 5 distict beer styles with the highest average score of smell + feel that were reviewed by the third most productive user. Keep in mind that cleaning the database earlier should ensure correct results.
5. Answer **two out of four** of the following questions using Graph Algorithms (gds): [NB: make sure to clear the graph before using it again] For the quarterly report, Analytics department the follownig information. [6 points]
    1. Which two countries are most similiar when it comes to their top five most produced Beer styles?
    2. Which beer is the most popular when considering the number of users who reviewed it? 
    3. Users are connected together by their reviews of beers, taking into consideration the "smell" score they assign as a weight, how many communities are formed from these relationships? How many users are in the three largest communities? 
    4. Which user is the most influential when it comes to reviews of distinct beers by style?
 

## Loading the Database


In [1]:
from neo4j import GraphDatabase
from pprint import pprint

In [2]:
NEO4J_URI="neo4j://localhost:7687"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="test"

In [3]:
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD), )

In [4]:
def execute_read(driver, query):    
    with driver.session(database="neo4j") as session:
        result = session.execute_read(lambda tx, query: list(tx.run(query)), query)
    return result

In [5]:
def execute_write(driver, query):
    with driver.session(database="neo4j") as session:
        # Write transactions allow the driver to handle retries and transient errors
        result = session.execute_write(lambda tx, query: list(tx.run(query)), query)
    return result

# Question 1: Exploring the database

## 1.1 Labels, Relationships and Properties

> Let's start by checking the labels in our database:

In [8]:
query = """
       call db.labels();
       """

result = execute_read(driver, query)

pprint(result)

[<Record label='COUNTRIES'>,
 <Record label='CITIES'>,
 <Record label='BREWERIES'>,
 <Record label='BEERS'>,
 <Record label='REVIEWS'>,
 <Record label='STYLE'>,
 <Record label='USER'>]


In [14]:
labels = ['COUNTRIES','CITIES','BREWERIES','BEERS','REVIEWS','STYLE','USER']

> Check the number of nodes for each label

In [15]:
for label in labels:
    query = f"""
        MATCH (n:{label})
        RETURN COUNT(n) AS count
        """
    
    result = execute_read(driver, query)
    print(f"Label: {label}, Count: {result[0]['count']}")


Label: COUNTRIES, Count: 400
Label: CITIES, Count: 23330
Label: BREWERIES, Count: 100694
Label: BEERS, Count: 417746
Label: REVIEWS, Count: 2549271
Label: STYLE, Count: 113
Label: USER, Count: 123935


> Relationships:

In [16]:
query = """
        CALL db.relationshipTypes();
       """

result = execute_read(driver, query)

pprint(result)

[<Record relationshipType='REVIEWED'>,
 <Record relationshipType='BREWED'>,
 <Record relationshipType='IN'>,
 <Record relationshipType='HAS_STYLE'>,
 <Record relationshipType='POSTED'>]


In [18]:
relationships = ['REVIEWED','BREWED','IN','HAS_STYLE','POSTED']

In [19]:
for relationship in relationships:
    query = f"""
        MATCH ()-[r:{relationship}]->()
        RETURN COUNT(r) AS count
        """
    
    result = execute_read(driver, query)
    print(f"Relationship: {relationship}, Count: {result[0]['count']}")


Relationship: REVIEWED, Count: 2537991
Relationship: BREWED, Count: 358873
Relationship: IN, Count: 62424
Relationship: HAS_STYLE, Count: 358873
Relationship: POSTED, Count: 2538044


> Properties:

In [20]:
query = """
        CALL db.propertyKeys();
        """

result = execute_read(driver, query)

pprint(result)

[<Record propertyKey='name'>,
 <Record propertyKey='types'>,
 <Record propertyKey='notes'>,
 <Record propertyKey='state'>,
 <Record propertyKey='id'>,
 <Record propertyKey='abv'>,
 <Record propertyKey='retired'>,
 <Record propertyKey='availability'>,
 <Record propertyKey='brewery_id'>,
 <Record propertyKey='date'>,
 <Record propertyKey='score'>,
 <Record propertyKey='taste'>,
 <Record propertyKey='feel'>,
 <Record propertyKey='overall'>,
 <Record propertyKey='beer_id'>,
 <Record propertyKey='text'>,
 <Record propertyKey='smell'>,
 <Record propertyKey='look'>]


In [21]:
properties = ['name','types','notes','state','id','abv','retired','availability',\
              'brewery_id','date','score','taste','feel','overall','beer_id',\
              'text','smell','look']

## 1.2 What labels are connected by each relationship type?

> Let's take a look at the schema:

In [22]:
query = """
        CALL db.schema.visualization();
        """

result = execute_read(driver, query)
pprint(result)

[<Record nodes=[<Node element_id='-5' labels=frozenset({'REVIEWS'}) properties={'name': 'REVIEWS', 'indexes': ['id'], 'constraints': []}>, <Node element_id='-4' labels=frozenset({'BEERS'}) properties={'name': 'BEERS', 'indexes': ['id'], 'constraints': []}>, <Node element_id='-1' labels=frozenset({'COUNTRIES'}) properties={'name': 'COUNTRIES', 'indexes': ['name'], 'constraints': []}>, <Node element_id='-3' labels=frozenset({'BREWERIES'}) properties={'name': 'BREWERIES', 'indexes': ['id'], 'constraints': []}>, <Node element_id='-6' labels=frozenset({'STYLE'}) properties={'name': 'STYLE', 'indexes': ['name'], 'constraints': []}>, <Node element_id='-2' labels=frozenset({'CITIES'}) properties={'name': 'CITIES', 'indexes': ['name'], 'constraints': []}>, <Node element_id='-7' labels=frozenset({'USER'}) properties={'name': 'USER', 'indexes': ['name'], 'constraints': []}>] relationships=[<Relationship element_id='-1' nodes=(<Node element_id='-4' labels=frozenset({'BEERS'}) properties={'name': '

> ADD IMAGE OF THE SCHEMA

> By looking at the schema in the localhost we can see that there are these relationships:<br>
> [BREWERIES]--'BREWED'-->[BEERS]<br>
> [BEERS]--'HAS_STYLE'-->[STYLE]<br>
> [CITIES]--'IN'-->[COUNTRIES]<br>
> [CITIES]--'IN'-->[CITIES]<br>
> [BREWERIES]--'IN'-->[COUNTRIES]<br>
> [BREWERIES]--'IN'-->[CITIES]<br>
> [REVIEWS]--'POSTED'-->[USER]<br>
> [BEERS]--'REVIEWED'-->[REVIEWS]

In [23]:
query = """
        MATCH (a)-[r]->(b)
        RETURN DISTINCT labels(a) AS Start, type(r) AS Relationship, labels(b) AS End, COUNT(r)
        ORDER BY Relationship;
        """

result = execute_read(driver, query)

pprint(result)

[<Record Start=['BREWERIES'] Relationship='BREWED' End=['BEERS'] COUNT(r)=358873>,
 <Record Start=['BEERS'] Relationship='HAS_STYLE' End=['STYLE'] COUNT(r)=358873>,
 <Record Start=['CITIES'] Relationship='IN' End=['COUNTRIES'] COUNT(r)=12077>,
 <Record Start=['BREWERIES'] Relationship='IN' End=['CITIES'] COUNT(r)=50347>,
 <Record Start=['REVIEWS'] Relationship='POSTED' End=['USER'] COUNT(r)=2538044>,
 <Record Start=['BEERS'] Relationship='REVIEWED' End=['REVIEWS'] COUNT(r)=2537991>]


> But when we check for existing relationships in the our database and their count, the relationships [CITIES]--'IN'-->[CITIES] and [BREWERIES]--'IN'-->[COUNTRIES] don't show up, and as we can see below, there are no instances of these relationships.

In [24]:
query = """
        MATCH (c1:CITIES)-[r:IN]->(c2:CITIES) RETURN COUNT(r);
        """

result = execute_read(driver, query)

pprint(result)

[<Record COUNT(r)=0>]


In [25]:
query = """
        MATCH (c1:BREWERIES)-[r:IN]->(c2:COUNTRIES) RETURN COUNT(r);
        """

result = execute_read(driver, query)

pprint(result)

[<Record COUNT(r)=0>]


> Not sure if it makes sense to have <br>
[REVIEWS]--POSTED-->[USER] and <br>
[BEERS]--'REVIEWED-->[REVIEWES].<br>
We should look into reversing the direction of these relationships. Or changing the relationship names to REVIEWED_BY and POSTED_BY.

## 1.3 What properties does each label have?

In [26]:
query = """
        CALL db.schema.nodeTypeProperties()
        YIELD nodeType, propertyName
        RETURN nodeType AS Label, COLLECT(propertyName) AS Properties
        ORDER BY Label;
        """

result = execute_read(driver, query)

pprint(result)

[<Record Label=':`BEERS`' Properties=['name', 'notes', 'state', 'id', 'abv', 'retired', 'availability', 'brewery_id']>,
 <Record Label=':`BREWERIES`' Properties=['name', 'types', 'notes', 'state', 'id']>,
 <Record Label=':`CITIES`' Properties=['name']>,
 <Record Label=':`COUNTRIES`' Properties=['name']>,
 <Record Label=':`REVIEWS`' Properties=['id', 'date', 'score', 'taste', 'feel', 'overall', 'beer_id', 'text', 'smell', 'look']>,
 <Record Label=':`STYLE`' Properties=['name']>,
 <Record Label=':`USER`' Properties=['name']>]


> Let's just take a look at the max and min values of reviews:

In [ ]:
review_scores = ['score', 'taste', 'feel', 'overall','smell', 'look']

for score in review_scores:
    query = f"""
        MATCH (r:REVIEWS)
        WHERE r.{score} <> 'nan' AND r.{score} IS NOT NULL
        RETURN 
            MIN(TOFLOAT(r.{score})) AS min_{score}, 
            MAX(TOFLOAT(r.{score})) AS max_{score}
    """
    result = execute_read(driver, query)
    print(f"Score: {score}")
    pprint(result)
    print("\n\n")


Score: score
[<Record min_score=1.0 max_score=5.0>]



Score: taste
[<Record min_taste=1.0 max_taste=5.0>]



Score: feel
[<Record min_feel=1.0 max_feel=5.0>]



Score: overall
[<Record min_overall=1.0 max_overall=5.0>]



Score: smell
[<Record min_smell=1.0 max_smell=5.0>]



Score: look
[<Record min_look=1.0 max_look=5.0>]





## 1.4 Indexes

> Let's check what indexes there are:

In [10]:
query = """
SHOW INDEXES;
       """

result = execute_read(driver, query)

pprint(result)

[<Record id=8 name='index_268c309e' state='ONLINE' populationPercent=100.0 uniqueness='NONUNIQUE' type='BTREE' entityType='NODE' labelsOrTypes=['STYLE'] properties=['name'] indexProvider='native-btree-1.0'>,
 <Record id=1 name='index_343aff4e' state='ONLINE' populationPercent=100.0 uniqueness='NONUNIQUE' type='LOOKUP' entityType='NODE' labelsOrTypes=None properties=None indexProvider='token-lookup-1.0'>,
 <Record id=3 name='index_476bd9ac' state='ONLINE' populationPercent=100.0 uniqueness='NONUNIQUE' type='BTREE' entityType='NODE' labelsOrTypes=['COUNTRIES'] properties=['name'] indexProvider='native-btree-1.0'>,
 <Record id=4 name='index_69b3ab56' state='ONLINE' populationPercent=100.0 uniqueness='NONUNIQUE' type='BTREE' entityType='NODE' labelsOrTypes=['CITIES'] properties=['name'] indexProvider='native-btree-1.0'>,
 <Record id=7 name='index_7d8f6eb8' state='ONLINE' populationPercent=100.0 uniqueness='NONUNIQUE' type='BTREE' entityType='NODE' labelsOrTypes=['REVIEWS'] properties=['id'

# Question 2

## 2.1 Isolated Nodes

> Let's start by removing isolated nodes

In [11]:
query = """
        MATCH (n)
        RETURN COUNT(n) AS total_nodes;
        """
result = execute_read(driver, query)
pprint(result)

[<Record total_nodes=3215489>]


In [7]:
query = """
        MATCH (n)
        WHERE NOT (n)-[]-()
        RETURN labels(n) AS NodeLabel, COUNT(n) AS isolated_node_count
        ORDER BY isolated_node_count DESC;
        """
result = execute_read(driver, query)
pprint(result)

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: } {title: This feature is deprecated and will be removed in future versions.} {description: Coercion of list to boolean is deprecated. Please consider using `NOT isEmpty(...)` instead.} {position: line: 3, column: 27, offset: 37} for query: '\n        MATCH (n)\n        WHERE NOT (n)-[]-()\n        RETURN labels(n) AS NodeLabel, COUNT(n) AS isolated_node_count\n        ORDER BY isolated_node_count DESC;\n        '


[<Record NodeLabel=['BEERS'] isolated_node_count=58873>,
 <Record NodeLabel=['BREWERIES'] isolated_node_count=50347>,
 <Record NodeLabel=['USER'] isolated_node_count=17290>,
 <Record NodeLabel=['CITIES'] isolated_node_count=11665>,
 <Record NodeLabel=['REVIEWS'] isolated_node_count=11208>,
 <Record NodeLabel=['COUNTRIES'] isolated_node_count=200>]


In [12]:
query = """
        MATCH (n)
        WHERE NOT (n)-[]-()
        DELETE n
        RETURN count(n);
        """
result = execute_write(driver, query)
pprint(result)

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: } {title: This feature is deprecated and will be removed in future versions.} {description: Coercion of list to boolean is deprecated. Please consider using `NOT isEmpty(...)` instead.} {position: line: 3, column: 27, offset: 37} for query: '\n        MATCH (n)\n        WHERE NOT (n)-[]-()\n        DELETE n\n        RETURN count(n);\n        '


[<Record count(n)=149583>]


In [9]:
query = """
        MATCH (n)
        RETURN COUNT(n) AS total_nodes;
        """
result = execute_read(driver, query)
pprint(result)

[<Record total_nodes=3065906>]


## 2.2 Unique values

> Let's take a look a the unique values of each property and check for inconsistencies:

In [13]:
node_properties = {
    "BEERS": ["notes", "name", "state", "id", "retired", "availability", "brewery_id"],
    "BREWERIES": ["notes", "types", "id", "name", "state"],
    "CITIES": ["name"],
    "COUNTRIES": ["name"],
    "REVIEWS": ["text", "smell", "look", "taste", "feel", "overall", "beer_id", "id", "date", "score"],
    "STYLE": ["name"],
    "USER": ["name"]
}

In [ ]:
for label, properties in node_properties.items():
    for property in properties:
        query = f"""
            MATCH (n:{label})
            RETURN DISTINCT n.{property} AS {property}, COUNT(n) AS Count
            ORDER BY Count DESC
            LIMIT 10
        """
        result = execute_read(driver, query)
        print(f"Label: {label}, Property: {property}")
        pprint(result)
        print("\n\n")

Label: BEERS, Property: notes
[<Record notes='No notes at this time.' Count=309078>,
 <Record notes='nan' Count=46>,
 <Record notes='Single-Hop IPA' Count=26>,
 <Record notes='Brewed at De Proefbrouwerij.' Count=24>,
 <Record notes=' No notes at this time.' Count=23>,
 <Record notes='30 IBU' Count=22>,
 <Record notes='Permutation is our experimental series of small batch offerings, showcasing the unique visions and innovative concepts developed by our brewing and cellar crew.' Count=19>,
 <Record notes='20 IBU' Count=19>,
 <Record notes='70 IBU' Count=19>,
 <Record notes='The Intervals series is a platform that allows our brewers to experience and study the ingredients that we use in brewing. As we develop new flavors and experience those nuances we can share that with others. From single hop varieties to alternate grains we want you to learn with us! Experiment, learn, repeat!' Count=18>]



Label: BEERS, Property: name
[<Record name='Oktoberfest' Count=755>,
 <Record name='IPA' Count

: 

### 2.2.1 Remove leading and trailing blank spaces

> **1.** We'll start by removing leading and trailing blank spaces. By checking first what properties have those blank spaces. Which are: BEERS[notes,availability] and BREWERIES[notes]

In [ ]:
for node, properties in node_properties.items():
    for prop in properties:
        print(f"Checking outter white spaces for: {prop} in {node}")
        query = f"""
                MATCH (n:{node})
                WHERE n.{prop} =~ "^\s.*|.*\s$"
                RETURN count(n) AS NodeCount
        """
        result = execute_read(driver, query)
        pprint(result)

Checking outter white spaces for: notes in BEERS


[<Record NodeCount=2287>]
Checking outter white spaces for: name in BEERS
[<Record NodeCount=0>]
Checking outter white spaces for: state in BEERS
[<Record NodeCount=0>]
Checking outter white spaces for: id in BEERS
[<Record NodeCount=0>]
Checking outter white spaces for: retired in BEERS
[<Record NodeCount=0>]
Checking outter white spaces for: availability in BEERS
[<Record NodeCount=248467>]
Checking outter white spaces for: brewery_id in BEERS
[<Record NodeCount=0>]
Checking outter white spaces for: notes in BREWERIES
[<Record NodeCount=315>]
Checking outter white spaces for: types in BREWERIES
[<Record NodeCount=0>]
Checking outter white spaces for: id in BREWERIES
[<Record NodeCount=0>]
Checking outter white spaces for: name in BREWERIES
[<Record NodeCount=0>]
Checking outter white spaces for: state in BREWERIES
[<Record NodeCount=0>]
Checking outter white spaces for: name in CITIES
[<Record NodeCount=0>]
Checking outter white spaces for: name in COUNTRIES
[<Record NodeCount=0>]
Ch

In [14]:
white_space_nodes_prop = {
    "BEERS": ["notes", "availability"],
    "BREWERIES": ["notes"]
}

for node, properties in white_space_nodes_prop.items():
    for prop in properties:
        print(f"Deleting outter white spaces for: {prop} in {node}")
        query = f"""
                MATCH (n:{node})
                WHERE n.{prop} =~ "^\s.*|.*\s$"
                SET n.{prop} = TRIM(n.{prop})
                RETURN count(n) AS NodeCount
        """
        result = execute_write(driver, query)
        pprint(result)

Deleting outter white spaces for: notes in BEERS
[<Record NodeCount=2287>]
Deleting outter white spaces for: availability in BEERS
[<Record NodeCount=248467>]
Deleting outter white spaces for: notes in BREWERIES
[<Record NodeCount=315>]


### 2.2.2 Remove '\xa0' from REVIEWS

> **2.** Remove '\xa0', which represents non breaking spaces, a space character that prevents line breaks. For a more consistent formatting.

In [ ]:
query = """
        MATCH (r:REVIEWS)
        WHERE r.text=~ '.*\\xa0.*' 
        RETURN COUNT(r) AS Count
"""
result = execute_read(driver, query)
pprint(result)

[<Record Count=2536660>]


In [17]:
loop = 0
while True:
    query = """
    MATCH (r:REVIEWS)
    WHERE r.text CONTAINS '\xa0'
    WITH r LIMIT 100000
    SET r.text = TRIM(REPLACE(r.text, '\xa0', ''))
    RETURN COUNT(r) AS Count
    """
    
    result = execute_write(driver, query)
    count = result[0]['Count']
    
    loop += 1
    pprint(f'Loop number {loop}. Cleaned {count} reviews.')

    if count == 0:
        break


'Loop number 1. Cleaned 100000 reviews.'
'Loop number 2. Cleaned 100000 reviews.'
'Loop number 3. Cleaned 100000 reviews.'
'Loop number 4. Cleaned 100000 reviews.'
'Loop number 5. Cleaned 100000 reviews.'
'Loop number 6. Cleaned 100000 reviews.'
'Loop number 7. Cleaned 100000 reviews.'
'Loop number 8. Cleaned 100000 reviews.'
'Loop number 9. Cleaned 100000 reviews.'
'Loop number 10. Cleaned 100000 reviews.'
'Loop number 11. Cleaned 100000 reviews.'
'Loop number 12. Cleaned 100000 reviews.'
'Loop number 13. Cleaned 100000 reviews.'
'Loop number 14. Cleaned 100000 reviews.'
'Loop number 15. Cleaned 100000 reviews.'
'Loop number 16. Cleaned 100000 reviews.'
'Loop number 17. Cleaned 100000 reviews.'
'Loop number 18. Cleaned 100000 reviews.'
'Loop number 19. Cleaned 100000 reviews.'
'Loop number 20. Cleaned 100000 reviews.'
'Loop number 21. Cleaned 100000 reviews.'
'Loop number 22. Cleaned 38044 reviews.'
'Loop number 23. Cleaned 0 reviews.'


In [8]:
query = """
        MATCH (r:REVIEWS)
        WHERE r.text=~ '.*\\xa0.*' 
        RETURN COUNT(r) AS Count
"""
result = execute_read(driver, query)
pprint(result)

[<Record Count=0>]


In [15]:
query = """
        MATCH (r:REVIEWS)
        RETURN r.text
        LIMIT 10;
"""
result = execute_read(driver, query)
pprint(result)

[<Record r.text=''>,
 <Record r.text=''>,
 <Record r.text='0% 16 oz can. Funny story: As I finally walked in the doors after a 45 min wait in line and freezing temps the sweet sound of the Grateful Dead\'s Sugar Magnolia greeted me from the TreeHouse sound system. The bottom of the can reads: "Going where the wind goes, bloomin\' like a red rose" A white haze to the yellow and golden liquid. Thick and healthy, totally unfiltered. Brawny white foam cap, thick, all-white clumps. Huge lacing left over. The aroma has a very zesty citrus hop effect, mellon and mango, grainy earthiness, tropical fruit blend with a bitter to sweet effect, then a peppery kick at the end. Very aromatic. The flavor is just bursting with complex hops, zesty earthy tones, sweet orange, peppery malt, clean fresh feel and overall vibe. A crispy bite wakes you up, full and lush mouthfeel follows from a totally unfiltered expereince. The feel and flavor finishes with a fun, earthy, zesty dry bite. Tropical juicy, zest

## 2.3 Check for inconsistencies and redundancies

### 2.3.1 Property state in BEERS and BREWERIES

> Both BREWERIES and BEERS have the property 'state', we'll check if the BEER and its' BREWER have the same state. 

> Each beer is brewed by only one brewery:

In [ ]:
# Check the number of beers that have been brewed by more than one brewery
query = """
    MATCH (a:BREWERIES)-[r:BREWED]->(b:BEERS)
    WITH b, COUNT(DISTINCT a) AS breweryCount
    WHERE breweryCount > 1
    RETURN COUNT(b) AS count
"""
result = execute_read(driver, query)
pprint(result)


[<Record count=0>]


> Only 132 have different values in state. Which are all the same cases: missing value in BEERS, but has value 'UT' in BREWERIES.

In [ ]:
#Check the number of beers and breweries that don't have the same state
query = """
    MATCH (a:BREWERIES)-[r:BREWED]->(b:BEERS)
    WHERE NOT a.state = b.state 
    RETURN a.state , b.state, Count(r) AS count
"""
result = execute_read(driver, query)
pprint(result)


[<Record a.state='UT' b.state='nan' count=134>]


> We'll change the 'nan' properties in BEERS to 'UT'. 

In [18]:
query = """
    MATCH (a:BREWERIES)-[r:BREWED]->(b:BEERS)
    WHERE b.state IS NULL OR a.state <> b.state
    SET b.state = a.state
    RETURN count(*) AS number_of_states_changed
"""
result = execute_write(driver, query)
pprint(result) 


[<Record number_of_states_changed=134>]


> It can be considered redundant to have the same property twice that saves the same information. We could turn it into a Label, but we'll leave it as it is.

### 2.3.2 Check for inconsistencies in brewery_id and beer_id

> As we have seen above, each beer is connected to only one brewery. But we should check if the brewery_id property of the beer is the same as the brewery to which it's connected to. As we can see, there are no inconsistencies:

In [ ]:
# Check if the brewery connected is the same as brewery_id
query = """
    MATCH (a:BREWERIES)-[r:BREWED]->(b:BEERS)
    WHERE b.brewery_id <> a.id
    OR b.brewery_id = 'nan' OR b.brewery_id IS NULL
    RETURN COUNT(r) AS count
"""
result = execute_read(driver, query)
pprint(result)

[<Record count=0>]


> Same as before, the beer_id on the REVIEW matches the beer that it is connected to:

In [ ]:
# Check if the beer connected to the review is the same as the beer_id
query = """
    MATCH (a:REVIEWS)<-[r:REVIEWED]-(b:BEERS)
    WHERE b.id <> a.beer_id
    RETURN COUNT(r) AS count
    """
result = execute_read(driver, query)
pprint(result)

[<Record count=0>]


> No inconsistencies here.

## 2.4 Missing values

In [ ]:
# We will start by checking NULL values and 'nan' values in all properties
nodes = ["BEERS", "BREWERIES", "CITIES", "COUNTRIES", "REVIEWS", "STYLE", "USER"]

for node in nodes: 
    query = f"""
            MATCH (n:{node})
            UNWIND keys(n) AS key
            WITH key
            WHERE n[key] IS NULL OR n[key] = 'nan'
            RETURN key, count(*) AS EmptyValuesCount
            ORDER BY key
    """
    result = execute_read(driver, query)
    print(f"Label: {node}")
    pprint(result)
    print("\n")

Label: BEERS
[<Record key='abv' EmptyValuesCount=38797>,
 <Record key='notes' EmptyValuesCount=46>,
 <Record key='state' EmptyValuesCount=60592>]


Label: BREWERIES
[<Record key='notes' EmptyValuesCount=85>,
 <Record key='state' EmptyValuesCount=11271>]


Label: CITIES
[<Record key='name' EmptyValuesCount=1>]


Label: COUNTRIES
[<Record key='name' EmptyValuesCount=1>]


Label: REVIEWS
[<Record key='feel' EmptyValuesCount=1060483>,
 <Record key='look' EmptyValuesCount=1060483>,
 <Record key='overall' EmptyValuesCount=1060483>,
 <Record key='smell' EmptyValuesCount=1060483>,
 <Record key='taste' EmptyValuesCount=1060483>]


Label: STYLE
[<Record key='name' EmptyValuesCount=1>]


Label: USER
[<Record key='name' EmptyValuesCount=1>]





> **BEERS**: There are missing values in abv (alcohol by volume) but we will leave it as it is. We changed some of the 'nan' values of state above, but there are still missing values, but we will leave it as it is too.

> **BREWERIES**: Missing values in state, but we'll leave it as it is. 

> **CITIES**: There is a 'nan' city connected to lots of different BREWERIES and COUNTRIES, we will remove this node, as it doesn't make sense for that many countries to be connected to it.

> **COUNTRIES**: There are 2 cities connected to a country with name 'nan', which are: Windhoek and Swakopmund. Both located in Namimbia, so we've decided to change it. 

> **STYLE**: Ony has one beer related to it: American Three Threads. We'll delete this node.

> **USER**: no name, but has reviews. We will leave it as it is.

> **REVIEWS**: has NULL values in some of the ratings, but it has values score.

> Convert all 'nan' to NULL, as converting it to null will remove those properties from the node and will save more space.


### 2.4.1 Missing values in CITIES

>There are 19 countries, and 58 breweries connected to the 'nan' city. We will remove this node, as it is most likely these relationships are wrong and it's an inconsistency.

In [ ]:
# Check for labels connected to the city with the name 'nan'
query = """
    MATCH (c:CITIES)-[r]-(b)
    WHERE c.name = 'nan'
    RETURN DISTINCT labels(b) AS connected_labels, COUNT(r) AS Count

    """
result = execute_read(driver, query)
pprint(result)


[<Record connected_labels=['BREWERIES'] Count=58>,
 <Record connected_labels=['COUNTRIES'] Count=19>]


In [19]:
# Delete the node
query = """
   MATCH (c:CITIES)
   WHERE c.name = 'nan'
   DETACH DELETE c
   RETURN COUNT(c) AS number_of_nan_cities
    """
result = execute_write(driver, query)
print(result)

[<Record number_of_nan_cities=1>]


### 2.4.2 Missing value in COUNTRY

In [ ]:
# Check for cities connected to the 'nan' country
query = """
            MATCH (c:COUNTRIES)--(b:CITIES)
            WHERE c.name = 'nan' OR c.name IS NULL
            RETURN b
    """
result = execute_read(driver, query)
pprint(result)


[<Record b=<Node element_id='9005' labels=frozenset({'CITIES'}) properties={'name': 'Swakopmund'}>>,
 <Record b=<Node element_id='4423' labels=frozenset({'CITIES'}) properties={'name': 'Windhoek'}>>]


> Both these cities belong to the country of Namibia.

In [20]:
# Change 'nan' country name to Namibia
query = f"""
        MATCH (c:COUNTRIES)
        WHERE c.name IS NULL or c.name = 'nan'
        SET c.name = 'Namibia'
    """
result = execute_write(driver, query)

In [ ]:
query = f"""
        MATCH (c:COUNTRIES)
        WHERE c.name='Namibia'
        RETURN c
    """
result = execute_read(driver, query)
pprint(result)

[<Record c=<Node element_id='133' labels=frozenset({'COUNTRIES'}) properties={'name': 'Namibia'}>>]


### 2.4.3 Missing value in STYLE

In [ ]:
# Check for beers connected to the 'nan' style
query = """
            MATCH (s:STYLE)--(b:BEERS)
            WHERE s.name = 'nan'
            RETURN b
    """
result = execute_read(driver, query)
pprint(result)


[<Record b=<Node element_id='269155' labels=frozenset({'BEERS'}) properties={'notes': 'No notes at this time.', 'abv': 'nan', 'name': 'American Three Threads', 'retired': 't', 'availability': 'Limited (brewed once)', 'brewery_id': '14', 'state': 'MA', 'id': '24360'}>>]


> There is only one beer connected to the Style with value 'nan', let's check if it is connected to any other beer, or if there are other beers with the same name.

In [ ]:
# Do beers have more than 1 style? 

query = """
        MATCH (b:BEERS)-[:HAS_STYLE]->(s:STYLE)
        WITH b, COUNT(s) AS styles_count
        WHERE styles_count > 1
        RETURN b.name AS beer_name, styles_count
    """
result = execute_read(driver, query)
pprint(result)

[]


> Each beer is connected to only 1 style.

In [ ]:
# Check for other beers with the name 'American Three Threads'
query = """
            MATCH (n:BEERS)
            WHERE n.name = 'American Three Threads'
            RETURN n
    """
result = execute_read(driver, query)
pprint(result)


[<Record n=<Node element_id='269155' labels=frozenset({'BEERS'}) properties={'notes': 'No notes at this time.', 'abv': 'nan', 'name': 'American Three Threads', 'retired': 't', 'availability': 'Limited (brewed once)', 'brewery_id': '14', 'state': 'MA', 'id': '24360'}>>]


> There only exists one beer named 'American Three Threads'. Since the 'nan' style node isn't providing any value, we will delete this node.

In [21]:
# Delete the style node with the name 'nan'
query = """
   MATCH (s:STYLE)
    WHERE s.name = 'nan'
    DETACH DELETE s
    RETURN COUNT(s) AS number_of_nan_styles;
    """
result = execute_write(driver, query)
pprint(result)

[<Record number_of_nan_styles=1>]


### 2.4.5 Set all 'nan' to NULL

In [22]:
nodes = ["BEERS", "BREWERIES", "USER"]

In [23]:
nan_values_dict = {
    "BEERS": ["abv", "notes", "state"],
    "BREWERIES": ["notes", "state"],
    "USER": ["name"]
}

for node, properties in nan_values_dict.items():
    for prop in properties:
        print(f"Correcting missing {prop} in {node}")
        query = f"""
                MATCH (n:{node})
                WHERE n.{prop} = 'nan'
                SET n.{prop} = NULL
                RETURN count(*) AS number_of_values_changed
        """
        result=execute_write(driver, query)
        pprint(result)

Correcting missing abv in BEERS
[<Record number_of_values_changed=38797>]
Correcting missing notes in BEERS
[<Record number_of_values_changed=46>]
Correcting missing state in BEERS
[<Record number_of_values_changed=60592>]
Correcting missing notes in BREWERIES
[<Record number_of_values_changed=85>]
Correcting missing state in BREWERIES
[<Record number_of_values_changed=11271>]
Correcting missing name in USER
[<Record number_of_values_changed=1>]


In [ ]:
# Set NULL the scores in reviews that have nan separatly as it takes a lot of memory to run
reviews_nan_prop = ["smell", "look", "taste", "feel", "overall", "score"]
query = f"""
        MATCH (n:REVIEWS)
        WHERE n.smell = 'nan' OR n.look = 'nan' OR n.taste = 'nan' OR n.feel = 'nan' OR n.overall = 'nan' OR n.score = 'nan'
        SET
                n.smell = CASE WHEN n.smell = 'nan' THEN NULL END,
                n.look = CASE WHEN n.look = 'nan' THEN NULL END,
                n.taste = CASE WHEN n.taste = 'nan' THEN NULL END,
                n.feel = CASE WHEN n.feel = 'nan' THEN NULL END,
                n.overall = CASE WHEN n.overall = 'nan' THEN NULL END,
                n.score = CASE WHEN n.score = 'nan' THEN NULL END
        RETURN count(*) AS number_of_values_changed
"""
result = execute_write(driver, query)
pprint(result)


Transaction failed and will be retried in 1.0651145572671588s (There is not enough memory to perform the current task. Please try increasing 'dbms.memory.heap.max_size' in the neo4j configuration (normally in 'conf/neo4j.conf' or, if you are using Neo4j Desktop, found through the user interface) or if you are running an embedded installation increase the heap by using '-Xmx' command line flag, and then restart the database.)


In [ ]:
# Set the review texts with no values to null
batch = 100_000
processed = 0
while True:
    query = f"""
            MATCH (n:REVIEWS)
            WHERE n.text = ''
            WITH n LIMIT {batch}
            SET n.text = NULL
            RETURN count(n) AS number_of_values_changed
    """
    result = execute_write(driver, query)
    if result[0]['number_of_values_changed'] == 0:
        break
    processed += result[0]['number_of_values_changed']
    print(f"Processed {processed} nodes")



> We will also set 'No noted at this time.' to nan, as it doesn't represent anything. And logically it would make more sense and saves space.

In [ ]:
query = f"""
    MATCH (n:BEERS)
    WHERE n.notes = "No notes at this time."
    SET n.notes = NULL
    RETURN count(*) AS number_of_values_changed
    """
result = execute_write(driver, query)
print(node)
pprint(result)

REVIEWS
[<Record number_of_values_changed=309097>]


## 2.5 Duplicate values

In [ ]:
unique_properties = ["id", "name"]

for node, properties in node_properties.items():
    for prop in unique_properties:
        if prop in properties: 
            print(f"Checking for duplicated values for '{prop}' in '{node}'")

            query = f"""
                    MATCH (n:{node})
                    WITH TRIM(n.{prop}) AS {prop}, count(n) AS count
                    WHERE count > 1
                    RETURN {prop}, count
                    ORDER BY count DESC
                    LIMIT 5
            """
            result = execute_read(driver, query)
            pprint(result)
            print("\n")
    print("-------------------------------")

Checking for duplicated values for 'id' in 'BEERS'
[]


Checking for duplicated values for 'name' in 'BEERS'
[<Record name='Oktoberfest' count=755>,
 <Record name='IPA' count=633>,
 <Record name='Pale Ale' count=620>,
 <Record name='Hefeweizen' count=477>,
 <Record name='Oatmeal Stout' count=443>]


-------------------------------
Checking for duplicated values for 'id' in 'BREWERIES'
[]


Checking for duplicated values for 'name' in 'BREWERIES'
[<Record name='Whole Foods Market' count=162>,
 <Record name='Total Wine & More' count=147>,
 <Record name='Cost Plus World Market' count=118>,
 <Record name='Mellow Mushroom' count=114>,
 <Record name="Trader Joe's" count=88>]


-------------------------------
Checking for duplicated values for 'name' in 'CITIES'
[]


-------------------------------
Checking for duplicated values for 'name' in 'COUNTRIES'
[]


-------------------------------
Checking for duplicated values for 'id' in 'REVIEWS'
[<Record id=None count=19>]


--------------------

> Duplicates of names in BEERS and BREWERIES is ok. 

> There are 19 reviews that have no properties and no relationships, of which we tried to delete but couldn't.

In [ ]:
#There are 19 REVIEWS with no id property and have no info associated
query = """
    MATCH (n:REVIEWS)
    WHERE n.id IS NULL
    RETURN n
"""
result = execute_read(driver, query)
pprint(result)


In [ ]:
#To remove the 19 reviews
query = """
    MATCH (n:REVIEWS)
    WHERE n.id IS NULL
    DELETE n
    RETURN count(n)
"""
result = execute_write(driver, query)
pprint(result)

[<Record count(n)=19>]


In [ ]:
# To see if they were deleted:
query = """
    MATCH (n:REVIEWS)
    WHERE n.id IS NULL
    RETURN n
"""
result = execute_read(driver, query)
pprint(result)

[<Record n=<Node element_id='921375' labels=frozenset() properties={}>>,
 <Record n=<Node element_id='921921' labels=frozenset() properties={}>>,
 <Record n=<Node element_id='922467' labels=frozenset() properties={}>>,
 <Record n=<Node element_id='923013' labels=frozenset() properties={}>>,
 <Record n=<Node element_id='923559' labels=frozenset() properties={}>>,
 <Record n=<Node element_id='924105' labels=frozenset() properties={}>>,
 <Record n=<Node element_id='924651' labels=frozenset() properties={}>>,
 <Record n=<Node element_id='925197' labels=frozenset() properties={}>>,
 <Record n=<Node element_id='925743' labels=frozenset() properties={}>>,
 <Record n=<Node element_id='926289' labels=frozenset() properties={}>>,
 <Record n=<Node element_id='926835' labels=frozenset() properties={}>>,
 <Record n=<Node element_id='927381' labels=frozenset() properties={}>>,
 <Record n=<Node element_id='927927' labels=frozenset() properties={}>>,
 <Record n=<Node element_id='928473' labels=frozens

## 2.6 Typecasting

In [6]:
query = """
        MATCH (n:BEERS)
        WHERE n.abv IS NOT NULL
        SET n.abv = toFloat(n.abv)
        RETURN COUNT(n) as Count
"""
res = execute_write(driver, query)
pprint(res)

[<Record Count=320076>]


In [ ]:


props = ["smell", "look", "taste", "feel", "overall", "score"]

batch = 50_000
processed_count = 0
while True: 
    query = f"""
            MATCH (n:REVIEWS)
            WITH n ORDER BY id(n) SKIP {processed_count} LIMIT {batch}
            WHERE n.{props[0]} IS NOT NULL or n.{props[1]} IS NOT NULL or n.{props[2]} IS NOT NULL or n.{props[3]} IS NOT NULL or n.{props[4]} IS NOT NULL or n.{props[5]} IS NOT NULL
            SET 
                    n.{props[0]} = CASE WHEN n.{props[0]} IS NOT NULL THEN toFloat(n.{props[0]}) END,
                    n.{props[1]} = CASE WHEN n.{props[1]} IS NOT NULL THEN toFloat(n.{props[1]}) END,
                    n.{props[2]} = CASE WHEN n.{props[2]} IS NOT NULL THEN toFloat(n.{props[2]}) END,
                    n.{props[3]} = CASE WHEN n.{props[3]} IS NOT NULL THEN toFloat(n.{props[3]}) END,
                    n.{props[4]} = CASE WHEN n.{props[4]} IS NOT NULL THEN toFloat(n.{props[4]}) END,
                    n.{props[5]} = CASE WHEN n.{props[5]} IS NOT NULL THEN toFloat(n.{props[5]}) END
            RETURN COUNT(n) as Count
    """
    res = execute_write(driver, query)
    
    if res[0]['Count'] == 0:
            break
        
    processed_count += res[0]['Count']        
    print(f"Processed {processed_count} nodes")
        

In [ ]:
# date format takes less space than string 
batch = 100_000
processed_count = 0
while True:
    query = f"""
            MATCH (n:REVIEWS)
            WHERE n.date IS NOT NULL
            WITH n ORDER BY id(n) SKIP {processed_count} LIMIT {batch}
            SET n.date = date(n.date)
            RETURN COUNT(n) as Count
    """
    res = execute_write(driver, query)
    
    if res[0]['Count'] == 0:
        break
    
    processed_count += res[0]['Count']
    print(f"Processed {processed_count} nodes")



In [ ]:

query = """
        MATCH (n:BEERS)
        WHERE n.retired IS NOT NULL
        RETURN COUNT(n) as Count
"""
res = execute_read(driver, query)
pprint(res)


In [ ]:
query = """
        MATCH (n:BEERS)
        RETURN COUNT(n) as Count
"""
res = execute_read(driver, query)
pprint(res)


In [ ]:
# true / false takes less space 
query = """
        MATCH (b:BEERS)
        WITH b,
        CASE 
            WHEN b.retired = 'f' THEN FALSE 
            WHEN b.retired = 't' THEN TRUE 
        END AS new_value
        SET b.retired = new_value
        RETURN COUNT(b) AS Count
"""
res = execute_write(driver, query)
pprint(res)


# Question 3

### 3.1 How many reviews has the beer with the most reviews?

In [37]:
query = """
MATCH (beer:BEERS)-[r:REVIEWED]->(:REVIEWS)
WITH beer, COUNT(r) AS rel_count
RETURN ID(beer), rel_count
ORDER BY rel_count DESC
LIMIT 1
    """

result = execute_read(driver, query)

beer_id = str(result[0]['rel_count'])
print(f"The beer with the most reviews has {beer_id} reviews")

: 

### 3.2 Which three users wrote the most reviews about beers?


In [15]:
query = """
MATCH (u:USER)-[p:POSTED]-(rev:REVIEWS)
WITH u, COUNT(p) as Nr_posted_reviews
RETURN u.name, Nr_posted_reviews
ORDER BY Nr_posted_reviews DESC
LIMIT 3
    """

result = execute_read(driver, query)

print("The three users that wrote the most reviews were:")
for i in result:
    users = str(i['u.name'])
    print(f'{users} posted {i["Nr_posted_reviews"]} reviews')

    

The three users that wrote the most reviews were:
Sammy posted 3756 reviews
acurtis posted 3403 reviews
kylehay2004 posted 3368 reviews


### 3.3 Find all beers that are described with following words: 'fruit', 'complex', 'nutty', 'dark'.

In [6]:
words = ["fruit", "complex", "nutty", "dark"]

query = f"""
        WITH {words} AS words
        
        MATCH (b:BEERS)
        WHERE all(word IN words WHERE toLower(b.notes) CONTAINS word)
        RETURN b.name, ID(b)
    """

result = execute_read(driver, query)

print("The ID and the name of the beers that are described using the words 'fruit', 'complex', 'nutty' and 'dark' are:")
for i in result:
    beer_ID = str(i['ID(b)'])
    beer_name =  str(i['b.name'])
    print(f"- {beer_name}, with ID {beer_ID}")

The ID and the name of the beers that are described using the words 'fruit', 'complex', 'nutty' and 'dark' are:
- Legalize Hemp Ale, with ID 160197
- Brother David's Belgian-style Double Ale, with ID 187422


### 3.4 Which top three breweries produce the largest variety of beer styles?

In [7]:
query = """
MATCH (br:BREWERIES)-[:BREWED]->(b:BEERS)-[:HAS_STYLE]->(s:STYLE)
WITH br, COUNT(DISTINCT(s)) AS VarietyBeerStyles
RETURN br.name, ID(br), VarietyBeerStyles
ORDER BY VarietyBeerStyles DESC
LIMIT 3
    """

result = execute_read(driver, query)

print("The ID and the name of the top three breweries that produce the largest variety of beer styles are:")
for i in result:
    brewery_ID = str(i['ID(br)'])
    brewery_name =  str(i['br.name'])
    print(f"- {brewery_name}, with ID {brewery_ID}")

The ID and the name of the top three breweries that produce the largest variety of beer styles are:
- Iron Hill Brewery & Restaurant, with ID 26055
- Rock Bottom Restaurant & Brewery, with ID 50012
- Goose Island Beer Co., with ID 45308


### 3.5 Which country produces the most beer styles?

In [8]:
query = """
MATCH (c:COUNTRIES)<-[:IN]-(:CITIES)<-[:IN]-(:BREWERIES)-[:BREWED]-(:BEERS)-[:HAS_STYLE]-(s:STYLE)
WITH c, COUNT(DISTINCT(s)) AS VarietyBeerStyles
RETURN c.name, VarietyBeerStyles
ORDER BY VarietyBeerStyles DESC
LIMIT 1
    """

result = execute_read(driver, query)

print(f"The country that produces the most beer styles is {result[0]['c.name']}.")

The country that produces the most beer styles is US.


# Question 4

### 4.1 Using ABV score, find five strongest beers, display their ABV score and the corresponding brewery? Keep in mind that the strongest known beer is Snake Venom, and deal with the error entries in the database.


In [15]:
# Check for the top 10 beers with the highest ABV
query = """
    MATCH (b:BEERS)
    WHERE b.abv IS NOT NULL
    RETURN b.name AS name, b.abv AS abv
    ORDER BY abv DESC
    LIMIT 10
"""
result = execute_read(driver, query)  
pprint(result)


[<Record name='Mysticum Cosmic Tripel' abv='9.99'>,
 <Record name='BLVCK Celebration - Oreo Cookie (Ghost 648)' abv='9.99'>,
 <Record name='Imperial Cascadian' abv='9.99'>,
 <Record name='BLVCK Celebration- Rum Barrel' abv='9.99'>,
 <Record name='Tanker/Mean Sardine Surf Wax' abv='9.99'>,
 <Record name="King's Jewels Double IPA" abv='9.99'>,
 <Record name='Double Justice' abv='9.99'>,
 <Record name='Bull Moose' abv='9.99'>,
 <Record name='BLVCK Celebration- Maple Whiskey' abv='9.99'>,
 <Record name='BLVCK Celebration: Dark Chocolate & Cocoa (Ghost 388)' abv='9.99'>]


> As it says in the question, the strangest known beer Snake Venom. So the other entries that are above Snake Venom should be considered as errors, and we'll procede to delete them.

In [ ]:
# Remove the beers with an abv greater than 67.5, and use DETACH as they have relationships with other nodes
query = """
    MATCH (b:BEERS)  
    WHERE b.abv > 67.5  
    DETACH DELETE b;
"""
result = execute_write(driver, query)

In [ ]:
# Let's take a look again at the top 10 beers with the highest ABV
query = """
    MATCH (b:BEERS)
    WHERE b.abv <> 'nan'
    RETURN b.name AS name, b.abv AS abv
    ORDER BY abv DESC
    LIMIT 10
"""
result = execute_read(driver, query)  
pprint(result)


> Our first try, which takes 818443 total db hits

In [ ]:
query = """
        PROFILE
        MATCH (b)<-[:BREWED]->(br:BREWERIES)
        WHERE b.abv <= 67.5
        RETURN b.name, b.abv, br.name
        ORDER BY b.abv DESC
        LIMIT 5
"""
result = execute_read(driver, query)
pprint(result)

> We tried to optimise it and obtained 717_764 total db hits

In [ ]:
query = """
        PROFILE
        MATCH (b:BEERS)  
        WHERE b.abv <= 67.5
        WITH b
            MATCH (b)<-[:BREWED]->(br:BREWERIES)
            RETURN b.name, b.abv, br.name
            ORDER BY b.abv DESC
            LIMIT 5
"""
result = execute_read(driver, query)
pprint(result)

### 4.2 Using the answer from question 2, find the top 5 distict beer styles with the highest average score of smell + feel that were reviewed by the third most productive user. Keep in mind that cleaning the database earlier should ensure correct results.

In [ ]:
orig_query = """
MATCH (u:USER)<-[p:POSTED]-(rev:REVIEWS)
WITH u, COUNT(p) as Nr_posted_reviews
ORDER BY Nr_posted_reviews DESC
SKIP 2 LIMIT 1
MATCH (u)<-[:POSTED]-(reviews:REVIEWS)<-[:REVIEWED]-(beers:BEERS)-[:HAS_STYLE]->(style:STYLE)
WITH style, AVG((toInteger(reviews.smell))+(toInteger(reviews.feel))) AS avg_score_smell_and_feel
WHERE avg_score_smell_and_feel IS NOT NULL
RETURN style.name AS styles, avg_score_smell_and_feel
ORDER BY avg_score_smell_and_feel DESC
LIMIT 5
    """

orig_result = execute_read(driver, orig_query)

pprint(orig_result)

#### Optimization

Optimization of the database

In [ ]:
'''
Precompute average scores and create a direct relationship from users to styles they reviewed with the score as a property
'''

query = """
MATCH (u:USER)<-[:POSTED]-(rev:REVIEWS)<-[:REVIEWED]-(beers:BEERS)-[:HAS_STYLE]->(style:STYLE)
WITH u, style, AVG(toInteger(rev.smell) + toInteger(rev.feel)) AS avg_smell_feel
MERGE (u)-[r:REVIEWED_STYLE]->(style)
SET r.avg_score_smell_feel = CASE WHEN avg_smell_feel IS NOT NULL THEN avg_smell_feel ELSE null END;
    """

result = execute_write(driver, query)

pprint(result)

Rewriting the query to answer the market analysis question

In [ ]:
query = """
MATCH (u:USER)<-[p:POSTED]-(rev:REVIEWS)
WITH u, COUNT(p) as Nr_posted_reviews
ORDER BY Nr_posted_reviews DESC
SKIP 2 LIMIT 1

MATCH (u)-[r:REVIEWED_STYLE]->(style:STYLE)
WHERE r.avg_score_smell_feel IS NOT NULL
RETURN style.name AS styles, r.avg_score_smell_feel AS avg_score_smell_feel
ORDER BY r.avg_score_smell_feel DESC
LIMIT 5
    """

result = execute_read(driver, query)

pprint(result)

#### Summary

- **QUERY PLAN** (using PROFILE through Neo4j browser): <br>

*Query Before Optimization (BO) Vs Query After Optimization (AO)*<br>

| Steps | BO - db hits | AO - db hits| BO - Memory(bytes) | AO - Memory(bytes) | BO - Filtered Rows | AO - Filtered Rows|
| --- | --- | --- | --- |---|---|---|
| 1| 123 936 | 123 936 |-|-|123 935|123 935|
| 2| 133 935 |133 935|- |-|10 000|10 000|
| 3| 10 000 | 10 000 |-|-|10 000|10 000|
| 4| 0 |0| 24|24|40|40|
|5| 0|0|728|728|3|3|
|6| 0 |0|-|-|1|1|
| 7| 826| 98 |-|-|825|97|
| 8| 825|194|-|-|825|97|
| 9| 1650|0|-|-|825|97|
| 10| 825|0|-|2 304|825|5|
| 11| 1650|5|-|-|825|5|
| 12| 825|0|-|-|825|5|
| 13| 1650|-|16 656|-|97|-|
| 14| 0|-|-|-|5|-|
| 15| 5|-|-|-|5|-|
|16|0| - |-|-|5|-|
| Total| 276 127 |  268 168  | 17 328|2 960|-|-|

<br>

| BO - Time(ms)| AO - Time(ms) |
| --- | --- |
| 12 704 | 734 |

*Precomputation - Auxiliary query for the database optimization*
| Time (ms) | db hits | Memory(bytes)|
|---|---|---|
|22 747| 389 817|16 768|

- **OPTIMIZATION PROCCESS**: In order to get a more efficient query, we decided to optimize the database with a direct relation from users to the styles they reviewed. In this relation we assign a property which corresponds to the average of the scores of smell and feel, that was precomputed when creating the relation between the nodes. This is also reducing the number of rows needed to be filtered. So, for the final query, the first part remained the same, with the objective to retrieve the third most productive user, and the second part changed accordingly to the optimized database.  <br>

- **CONCLUSION**: As we can observe in the tables, the query was succesfully optimized. As expected, the optimizated proccess is noticed in the second part of the query, starting at step 7, and we can clearly state that it takes less memory usage and db hits is reduced at each step and in total, all this resulting in less running time.
Now, the query in question can be runned several times daily, while reducing the pre-proposed computational costs.


# Question 5

### 5.1 Which two countries are most similiar when it comes to their top five most produced Beer styles?

In [61]:
try:
    query = """CALL gds.graph.drop('HM_1.1') YIELD graphName;  """
    result = execute_read(driver, query)
    pprint(result)
except Exception as e:
    pprint(e)

[<Record graphName='HM_1.1'>]


In [62]:
# INDEX QUERYS
    
try:
    query = """
            MATCH (n) 
            WHERE head(Labels(n))='COUNTRIES' or head(Labels(n))='STYLE' 
            RETURN id(n) as id 
            LIMIT 5 
        """
        #     RETURN distinct Labels (n) - to check
    result = execute_read(driver, query)
    pprint(result)
except Exception as e:
    pprint(e)

[<Record id=0>, <Record id=1>, <Record id=2>, <Record id=3>, <Record id=4>]


In [63]:
# RELATIONSHIP QUERIES 

try:
    query = """
            MATCH (c: COUNTRIES)--()--()--()-[:HAS_STYLE]->(s:STYLE)
            WITH distinct c, s.name as style, count(s) as freq 
                ORDER BY c.name, freq DESC
                WITH c, collect({style: style, frequency: freq})[..5] as top5
                WHERE size(top5) >= 5
                    UNWIND top5 AS style
                    RETURN c.name as source, style.style AS target, style.frequency AS freq 
        """
    result = execute_read(driver, query)
    pprint(result)
except Exception as e:
    pprint(e)



[<Record source='AD' target='American Blonde Ale' freq=2>,
 <Record source='AD' target='German Hefeweizen' freq=1>,
 <Record source='AD' target='English India Pale Ale (IPA)' freq=1>,
 <Record source='AD' target='Smoke Beer' freq=1>,
 <Record source='AD' target='German Doppelbock' freq=1>,
 <Record source='AG' target='American Pale Ale (APA)' freq=6>,
 <Record source='AG' target='Fruit and Field Beer' freq=4>,
 <Record source='AG' target='American Porter' freq=4>,
 <Record source='AG' target='American Light Lager' freq=4>,
 <Record source='AG' target='American Adjunct Lager' freq=4>,
 <Record source='AL' target='European Pale Lager' freq=8>,
 <Record source='AL' target='Bohemian Pilsener' freq=2>,
 <Record source='AL' target='American Amber / Red Lager' freq=1>,
 <Record source='AL' target='American Blonde Ale' freq=1>,
 <Record source='AL' target='German Maibock' freq=1>,
 <Record source='AM' target='European Pale Lager' freq=21>,
 <Record source='AM' target='Smoke Beer' freq=2>,
 <Re

In [64]:

try:
    query = """
            CALL gds.graph.project.cypher(
                "HM_1.1",
                
                "
                MATCH (n) 
                WHERE head(Labels(n))='COUNTRIES' or head(Labels(n))='STYLE' 
                RETURN id(n) as id
                ",
                
                "
                MATCH (c: COUNTRIES)--()--()--()-[:HAS_STYLE]->(s:STYLE)
                WITH distinct c, s, count(s) as freq 
                    ORDER BY c.name, freq DESC
                    WITH c, collect({name: s, frequency: freq})[..5] as top5
                    WHERE size(top5) >= 5
                        UNWIND top5 AS style
                        RETURN id(c) as source, id(style.name) AS target, style.frequency AS weight
                "
            )
    """
    result = execute_read(driver, query)
    pprint(result)
except Exception as e:
    pprint(e)
    

[<Record nodeQuery="MATCH (n) \n                WHERE head(Labels(n))='COUNTRIES' or head(Labels(n))='STYLE' \n                RETURN id(n) as id" relationshipQuery='MATCH (c: COUNTRIES)--()--()--()-[:HAS_STYLE]->(s:STYLE)\n                WITH distinct c, s, count(s) as freq \n                    ORDER BY c.name, freq DESC\n                    WITH c, collect({name: s, frequency: freq})[..5] as top5\n                    WHERE size(top5) >= 5\n                        UNWIND top5 AS style\n                        RETURN id(c) as source, id(style.name) AS target, style.frequency AS weight' graphName='HM_1.1' nodeCount=513 relationshipCount=705 projectMillis=6323>]


In [65]:
try:
    query = """
            CALL gds.nodeSimilarity.stream("HM_1.1", {relationshipWeightProperty:'weight'})
            YIELD node1, node2, similarity
            
            WITH gds.util.asNode(node1).name AS country1, gds.util.asNode(node2).name AS country2, similarity

                WHERE country1 < country2
                RETURN country1, country2, similarity
                ORDER BY similarity DESC
                LIMIT 10
    """
    result = execute_read(driver, query)
    pprint(result)
except Exception as e:
    pprint(e)
    

[<Record country1='CU' country2='ET' similarity=1.0>,
 <Record country1='ET' country2='nan' similarity=0.9988425925925926>,
 <Record country1='CU' country2='nan' similarity=0.9988425925925926>,
 <Record country1='SG' country2='nan' similarity=0.9840546697038725>,
 <Record country1='ET' country2='SG' similarity=0.9829157175398633>,
 <Record country1='CU' country2='SG' similarity=0.9829157175398633>,
 <Record country1='KY' country2='SG' similarity=0.7856420626895855>,
 <Record country1='KY' country2='UA' similarity=0.777120315581854>,
 <Record country1='KY' country2='nan' similarity=0.7750759878419453>,
 <Record country1='ET' country2='KY' similarity=0.7740628166160081>]


In [66]:
query = """
        MATCH (c:COUNTRIES)--()--()--()-[:HAS_STYLE]->(s:STYLE)
        WHERE c.name = "ET"
        RETURN c.name, s.name, count(*) AS freq
        ORDER BY c.name, freq DESC
        LIMIT 5
"""
result = execute_read(driver, query)
pprint(result)

[<Record c.name='ET' s.name='American IPA' freq=369>,
 <Record c.name='ET' s.name='American Pale Ale (APA)' freq=166>,
 <Record c.name='ET' s.name='American Porter' freq=122>,
 <Record c.name='ET' s.name='American Amber / Red Ale' freq=107>,
 <Record c.name='ET' s.name='American Imperial IPA' freq=99>]


In [67]:
query = """
        MATCH (c:COUNTRIES)--()--()--()-[:HAS_STYLE]->(s:STYLE)
        WHERE c.name = "CU"
        RETURN c.name, s.name, count(*) AS freq
        ORDER BY c.name, freq DESC
        LIMIT 5
"""
result = execute_read(driver, query)
pprint(result)

[<Record c.name='CU' s.name='American IPA' freq=369>,
 <Record c.name='CU' s.name='American Pale Ale (APA)' freq=166>,
 <Record c.name='CU' s.name='American Porter' freq=122>,
 <Record c.name='CU' s.name='American Amber / Red Ale' freq=107>,
 <Record c.name='CU' s.name='American Imperial IPA' freq=99>]


### 5.2 Which beer is the most popular when considering the number of users who reviewed it?  

In [ ]:
try:
    query = """
            CALL gds.graph.drop('exercise_5_2') YIELD graphName;
        """

    result = execute_read(driver, query)

    pprint(result)
except Exception as e:
    pprint(e)

ClientError('Failed to invoke procedure `gds.graph.drop`: Caused by: java.util.NoSuchElementException: Graph with name `exercise_5_2` does not exist on database `neo4j`. It might exist on another database.')


> Let's project the graph to get the connection between user and beers. Note that it is important to RETURN DISTINCT as there are users that have reviewed the same beer more than once. 

In [35]:
try:
    query = """
        CALL gds.graph.project.cypher(
            'exercise_5_2_2',
            
            'MATCH (b:BEERS) RETURN id(b) AS id
             UNION
             MATCH (u:USER) RETURN id(u) AS id',
            
            'MATCH (u:USER)<-[:POSTED]-(r:REVIEWS)<-[:REVIEWED]-(b:BEERS)
             RETURN DISTINCT id(u) AS target, id(b) AS source'
        )
    """
    result = execute_read(driver, query)
    pprint(result)
except Exception as e:
    pprint(e)


[<Record nodeQuery='MATCH (b:BEERS) RETURN id(b) AS id\n             UNION\n             MATCH (u:USER) RETURN id(u) AS id' relationshipQuery='MATCH (u:USER)<-[:POSTED]-(r:REVIEWS)<-[:REVIEWED]-(b:BEERS)\n             RETURN DISTINCT id(u) AS target, id(b) AS source' graphName='exercise_5_2_2' nodeCount=465518 relationshipCount=2537753 projectMillis=686528>]


> We will use Degree Centrality as it's better to measure the number of nodes directly connected. We want to know which one is more popular.

In [36]:
try:
        query = """
        CALL gds.degree.stream('exercise_5_2_2')

        YIELD nodeId, score

        RETURN gds.util.asNode(nodeId).name AS beer_name, score AS num_users,  gds.util.asNode(nodeId).id AS beer_id

        ORDER BY num_users DESC

        LIMIT 1
        """
        result = execute_read(driver, query)
        print(f'{result[0]["beer_name"]} (id={result[0]["beer_id"]}) is the most popular beer, with {result[0]["num_users"]} users reviewing it.')

except Exception as e:
    pprint(e)


Breakfast Stout (id=11757) is the most popular beer, with 4828.0 users reviewing it.


In [37]:

query = """
        MATCH (b:BEERS)--(r:REVIEWS)--(u:USER)
        WHERE b.id = '11757'
        RETURN COUNT(DISTINCT u) AS num_users

        """
result = execute_read(driver, query)
print(result)

[<Record num_users=4828>]


### 5.3 Users are connected together by their reviews of beers, taking into consideration the "smell" score they assign as a weight, how many communities are formed from these relationships? How many users are in the three largest communities? 

> For this question we've decided to use the Label Propagation Algorithm as shown in the practical class.

<strong style='color:red'>I have to check this again</strong>

> To start with, we need to project a subgraph that contains only the relationships that are relevant to our question. We need to get a graph that connects users to each other through the beers they review, but there isn’t a direct relationship between the users. So we’ll have to connect them through: User --> Posted -->Reviews and Reviews --> Reviewed --> Beers <br>
> Note that the smell must we used as weight!

In [ ]:
try:
    query = """
            CALL gds.graph.drop('exercise_5_3') YIELD graphName;
        """

    result = execute_read(driver, query)

    pprint(result)
except Exception as e:
    pprint(e)

[<Record graphName='exercise_5_3'>]


In [ ]:
query = """
        CALL gds.graph.project.cypher(
        
            'exercise_5_3',
            
            'MATCH (n:USER) RETURN id(n) AS id',
            
            'MATCH (u1:USER)-[:POSTED]-(r1:REVIEWS)-[:REVIEWED]-(b:BEERS)
            -[:REVIEWED]-(r2:REVIEWS)-[:POSTED]-(u2:USER) 
            
            WHERE id(u1)<id(u2) AND r1.beer_id=r2.beer_id
            RETURN id(u1) AS source, id(u2) AS target, (1 / (1 + abs(toFloat(r1.smell) - toFloat(r2.smell)))) AS weight
            '
        )
        """

result = execute_read(driver, query)

pprint(result)


[<Record nodeQuery='MATCH (n:USER) RETURN id(n) AS id' relationshipQuery='MATCH (u1:USER)<-[:POSTED]-(r1:REVIEWS)<-[:REVIEWED]-(b:BEERS)\n            -[:REVIEWED]->(r2:REVIEWS)-[:POSTED]->(u2:USER) \n            \n            WHERE id(u1)<id(u2) AND r1.beer_id = r2.beer_id\n            RETURN id(u1) AS source, id(u2) AS target, (1 / (1 + abs(toFloat(r1.smell) - toFloat(r2.smell)))) AS weight\n            LIMIT 100000' graphName='exercise_5_3' nodeCount=106645 relationshipCount=100000 projectMillis=6202>]


> We use the difference between the score that the users give to smell as weight. Since the higher the weight, then more similar the users are. As they have given a similar score to smell.

In [25]:
#Find the total number of communities:
query = """
    CALL gds.labelPropagation.stream('exercise_5_3')
    YIELD communityId AS Community
    RETURN count(distinct Community) AS number_of_communities
"""

result = execute_read(driver, query)

pprint(result)


[<Record number_of_communities=105347>]


> From here, we get that there are ---- communities

In [ ]:
# Find the top 3 largest communities, and the number of users:
query = """
            CALL gds.labelPropagation.stream('exercise_5_3')
            YIELD nodeId, communityId AS Community
    
            WITH gds.util.asNode(nodeId).name AS Name, Community
            RETURN Community, count(*) as number_of_users
            ORDER BY community_size desc
            LIMIT 3
        """

result = execute_read(driver, query)

pprint(result)


[<Record Community=9548119 community_size=736>,
 <Record Community=9548053 community_size=235>,
 <Record Community=9547932 community_size=90>]


> LIMIT 3 to get top 3, since we are ordering it in descending order. 